<a href="https://colab.research.google.com/github/Anil1331/EIP3-Assignments/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/'My Drive'/wonderland.txt .

In [0]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

Using TensorFlow backend.


In [0]:
raw_text[:1000]

'alices adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 30\n\n\n\n\nchapter i down the rabbithole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank and of having nothing to do once or twice she had peeped into the\nbook her sister was reading but it had no pictures or conversations in\nit and what is the use of a book thought alice without pictures or\nconversations\n\nso she was considering in her own mind as well as she could for the\nhot day made her feel very sleepy and stupid whether the pleasure\nof making a daisychain would be worth the trouble of getting up and\npicking the daisies when suddenly a white rabbit with pink eyes ran\nclose by her\n\nthere was nothing so very remarkable in that nor did alice think it so\nvery much out of the way to hear the rabbit say to itself oh dear\noh dear i shall be late when she thought it over afterwards it\noccurred to her that she ought to have wondered at this but at the time\nit al

Removed Punctuation from source text for better results

In [0]:
import re
raw_text = re.sub(r'[^\w\s]','',raw_text)

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
chars

['\n',
 ' ',
 '0',
 '3',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  136091
Total Vocab:  31


Article is split by sentences and each set of 100 characters in a sentence is used for training. If the length of sentence is less than 100 it is **padded** using keras **pad_sequences**

In [0]:
sentence = raw_text.split("\n\n")
from keras.preprocessing.sequence import pad_sequences
sent = ([x for x in sentence if sum(c.isalpha() for c in x)>1])
dataX = []
dataY = []
seq_length = 101
for s in sent:
  if len(s)<101:
    x = [char_to_int[a] for a in s]
    x = pad_sequences([x], maxlen=seq_length)
    dataX.append([a for a in x[0][:-1]])
    dataY.append(x[0][-1])
  else:
    for i in range(0, len(s) - seq_length, 1):
      x = [char_to_int[a] for a in s[i:i + seq_length]] 
      dataX.append([a for a in x[:-1]])
      dataY.append(x[-1])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
print(len(dataX))
X = numpy.reshape(numpy.array(dataX), (n_patterns, seq_length-1, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)      

Total Patterns:  68970
68970


In [0]:
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
# fit the model
model.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

Epoch 1/100
68970/68970 [==============================] - 118s 2ms/step - loss: 2.8818

Epoch 00001: loss improved from inf to 2.88184, saving model to weights-improvement-01-2.8818-bigger.hdf5
Epoch 2/100
68970/68970 [==============================] - 116s 2ms/step - loss: 2.8148

Epoch 00002: loss improved from 2.88184 to 2.81480, saving model to weights-improvement-02-2.8148-bigger.hdf5
Epoch 3/100
68970/68970 [==============================] - 115s 2ms/step - loss: 2.7083

Epoch 00003: loss improved from 2.81480 to 2.70829, saving model to weights-improvement-03-2.7083-bigger.hdf5
Epoch 4/100
68970/68970 [==============================] - 116s 2ms/step - loss: 2.6400

Epoch 00004: loss improved from 2.70829 to 2.63997, saving model to weights-improvement-04-2.6400-bigger.hdf5
Epoch 5/100
68970/68970 [==============================] - 114s 2ms/step - loss: 2.5608

Epoch 00005: loss improved from 2.63997 to 2.56079, saving model to weights-improvement-05-2.5608-bigger.hdf5
Epoch 6/1

In [0]:
import sys
int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" ant the king said turning to the jury they were
just beginning to write this down on their slates wh "
ar tueer lls tenemberer i an io tie wooold to the whe polee of teach whe suean whe gatdr anlce sectidd ooch oarty anlce whe eoow suier siolle bnice whe polent thalint anice coulde of gettet alice began toall be a large and tooetneer what suehcat sfmeiicered tound and look d ano turt i tuappee ier anice cooidd awatp yhat iad mapied becurifat ofx anai iopedi and teadhinid iir head tevplces k onled at all oergatunane yhich satpintsy anice teadi tomething anice loowes at ier tie whe polehteoed ou te
Done.
